<a href="https://colab.research.google.com/github/Nehal0952/-Explainable-AI-labAssignment-/blob/main/Ass_5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

df = pd.read_csv("/content/annual-co2-emissions-per-country.csv")  # Replace with actual path

# Basic inspection
print(df.shape)
print(df.dtypes)
print(df.isnull().sum())

# Summary stats
print(df.describe())

# Time trends
sns.lineplot(data=df, x='Year', y='Annual CO2 emissions')
plt.title("CO₂ Emissions Over Time by Region")
plt.show()


FileNotFoundError: [Errno 2] No such file or directory: '/content/annual-co2-emissions-per-country.csv'

In [ ]:
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split

# Rename columns
df.rename(columns={'Entity': 'Country', 'Code': 'Region'}, inplace=True)

# Handle missing values in numeric columns
for col in df.select_dtypes(include='number').columns:
  df[col].fillna(df[col].median(), inplace=True)

# Encode categorical variables
df['Country'] = LabelEncoder().fit_transform(df['Country'])
df['Region'] = LabelEncoder().fit_transform(df['Region'])

# Scale features
features = df.drop(columns=['Annual CO2 emissions'])
target = df['Annual CO2 emissions']
scaler = StandardScaler()
X_scaled = scaler.fit_transform(features)

# Train/test split
X_train, X_test, y_train, y_test = train_test_split(X_scaled, target, test_size=0.2, random_state=42)

In [ ]:
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.svm import SVR
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import numpy as np

models = {
    "Linear": LinearRegression(),
    "Ridge": Ridge(),
    "Lasso": Lasso(),
    "DecisionTree": DecisionTreeRegressor(),
    "RandomForest": RandomForestRegressor(),
    "GradientBoosting": GradientBoostingRegressor(),
    "SVR": SVR()
}

for name, model in models.items():
    model.fit(X_train, y_train)
    preds = model.predict(X_test)
    mse = mean_squared_error(y_test, preds)
    rmse = np.sqrt(mse)
    print(f"{name} → MAE: {mean_absolute_error(y_test, preds):.2f}, MSE: {mse:.2f}, RMSE: {rmse:.2f}, R²: {r2_score(y_test, preds):.2f}")


In [3]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv1D, Flatten, LSTM, GRU, MaxPooling1D
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import numpy as np

# MLP
mlp = Sequential([
    Dense(128, activation='relu', input_shape=(X_train.shape[1],)),
    Dense(64, activation='relu'),
    Dense(1)
])
mlp.compile(optimizer='adam', loss='mse', metrics=['mae'])
print("Training MLP...")
mlp.fit(X_train, y_train, epochs=50, validation_split=0.2, verbose=0)
mlp_preds = mlp.predict(X_test)
mlp_mse = mean_squared_error(y_test, mlp_preds)
mlp_rmse = np.sqrt(mlp_mse)
print(f"MLP → MAE: {mean_absolute_error(y_test, mlp_preds):.2f}, MSE: {mlp_mse:.2f}, RMSE: {mlp_rmse:.2f}, R²: {r2_score(y_test, mlp_preds):.2f}")

# CNN (1D)
X_train_seq_cnn = X_train.reshape(X_train.shape[0], X_train.shape[1], 1)
X_test_seq_cnn = X_test.reshape(X_test.shape[0], X_test.shape[1], 1)
cnn = Sequential([
    Conv1D(64, 1, activation='relu', input_shape=(X_train_seq_cnn.shape[1], 1)),
    Flatten(),
    Dense(1)
])
cnn.compile(optimizer='adam', loss='mse')
print("Training CNN...")
cnn.fit(X_train_seq_cnn, y_train, epochs=50, validation_split=0.2, verbose=0)
cnn_preds = cnn.predict(X_test_seq_cnn)
cnn_mse = mean_squared_error(y_test, cnn_preds)
cnn_rmse = np.sqrt(cnn_mse)
print(f"CNN → MAE: {mean_absolute_error(y_test, cnn_preds):.2f}, MSE: {cnn_mse:.2f}, RMSE: {cnn_rmse:.2f}, R²: {r2_score(y_test, cnn_preds):.2f}")


# LSTM
X_train_seq_lstm = X_train.reshape(X_train.shape[0], X_train.shape[1], 1)
X_test_seq_lstm = X_test.reshape(X_test.shape[0], X_test.shape[1], 1)
lstm = Sequential([
    LSTM(64, input_shape=(X_train_seq_lstm.shape[1], 1)),
    Dense(32, activation='relu'),
    Dense(1)
])
lstm.compile(optimizer='adam', loss='mse')
print("Training LSTM...")
lstm.fit(X_train_seq_lstm, y_train, epochs=50, validation_split=0.2, verbose=0)
lstm_preds = lstm.predict(X_test_seq_lstm)
lstm_mse = mean_squared_error(y_test, lstm_preds)
lstm_rmse = np.sqrt(lstm_mse)
print(f"LSTM → MAE: {mean_absolute_error(y_test, lstm_preds):.2f}, MSE: {lstm_mse:.2f}, RMSE: {lstm_rmse:.2f}, R²: {r2_score(y_test, lstm_preds):.2f}")

NameError: name 'X_train' is not defined

In [ ]:
import shap
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.inspection import PartialDependenceDisplay

# SHAP
explainer = shap.Explainer(models["GradientBoosting"], X_train)
shap_values = explainer(X_test)
shap.summary_plot(shap_values, features)

# LIME
import lime
import lime.lime_tabular
lime_explainer = lime.lime_tabular.LimeTabularExplainer(X_train, feature_names=features.columns.tolist(), mode='regression')
lime_exp = lime_explainer.explain_instance(X_test[0], models["RandomForest"].predict)
lime_exp.show_in_notebook()

In [ ]:
import shap
from sklearn.ensemble import GradientBoostingRegressor
# from sklearn.inspection import PartialDependenceDisplay # Removed due to error

# SHAP
explainer = shap.Explainer(models["GradientBoosting"], X_train)
shap_values = explainer(X_test)
shap.summary_plot(shap_values, features)

# PDP # Removed due to error
# PartialDependenceDisplay.from_estimator(models["RandomForest"], X_test, ['Coal_Emissions'])
# plt.show()

# LIME
import lime
import lime.lime_tabular
lime_explainer = lime.lime_tabular.LimeTabularExplainer(X_train, feature_names=features.columns.tolist(), mode='regression')
lime_exp = lime_explainer.explain_instance(X_test[0], models["RandomForest"].predict)
lime_exp.show_in_notebook()

In [ ]:
import matplotlib.pyplot as plt
from sklearn.inspection import PartialDependenceDisplay

# PDP
# We will use the 'Year' feature for the PDP plot as it is a relevant numerical feature.
PartialDependenceDisplay.from_estimator(models["RandomForest"], X_test, ['Year'], feature_names=features.columns.tolist())
plt.suptitle("Partial Dependence Plot for Year")
plt.show()